In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2

In [4]:
import numpy as np

In [8]:
import cv2
import numpy as np

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    # Define Region of Interest (ROI)
    roi = frame[100:300, 100:300]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define HSV range for skin color
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Create skin mask
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=4)
    mask = cv2.GaussianBlur(mask, (5, 5), 100)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    gesture_text = ""  # Default gesture

    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(roi, [max_contour], -1, (0, 255, 0), 2)

        hull = cv2.convexHull(max_contour)
        cv2.drawContours(roi, [hull], -1, (0, 0, 255), 2)

        hull_indices = cv2.convexHull(max_contour, returnPoints=False)
        if hull_indices is not None and len(hull_indices) > 3:
            defects = cv2.convexityDefects(max_contour, hull_indices)

            if defects is not None:
                count = 0
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i, 0]
                    start = tuple(max_contour[s][0])
                    end = tuple(max_contour[e][0])
                    far = tuple(max_contour[f][0])

                    a = np.linalg.norm(np.array(end) - np.array(start))
                    b = np.linalg.norm(np.array(far) - np.array(start))
                    c = np.linalg.norm(np.array(end) - np.array(far))
                    # Use cosine rule: cos(angle) = (b^2 + c^2 - a^2) / (2*b*c)
                    cos_angle = (b*b + c*c - a*a) / (2 * b * c + 1e-8)
                    cos_angle = np.clip(cos_angle, -1.0, 1.0)
                    angle = np.arccos(cos_angle)

                    if angle <= np.pi / 2:
                        count += 1
                        cv2.circle(roi, far, 5, [255, 0, 0], -1)

                # Map count to text
                if count == 0:
                    gesture_text = "ONE"
                elif count == 1:
                    gesture_text = "TWO"
                elif count == 2:
                    gesture_text = "THREE"
                elif count == 3:
                    gesture_text = "FOUR"
                elif count == 4:
                    gesture_text = "FIVE"
                else:
                    gesture_text = "UNKNOWN"

                # Display gesture name
                cv2.putText(frame, f"Gesture: {gesture_text}", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Draw rectangle around ROI
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)

    # Show the output
    cv2.imshow("Skin Mask", mask)
    cv2.imshow("Gesture", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 